In [ ]:
import os
import sys

org_wd = os.getcwd()
os.chdir('/home/smf19/creditfraud/creditfraud')
!{sys.executable} -m pip install -r /home/smf19/creditfraud/creditfraud/requirements.txt

os.chdir(org_wd)

Loading Packages

In [2]:
import json
import re
import sys
import subprocess
import os

def install_packages_from_notebook(notebook_path):
    # Load the notebook content
    with open(notebook_path, 'r') as f:
        notebook = json.load(f)

    # Extract code cells
    code_cells = [cell for cell in notebook['cells'] if cell['cell_type'] == 'code']

    # Extract import statements
    imports = set()
    for cell in code_cells:
        for line in cell['source']:
            match = re.match(r'^\s*import\s+(\w+)', line)
            if match:
                imports.add(match.group(1))
            match = re.match(r'^\s*from\s+(\w+)', line)
            if match:
                imports.add(match.group(1))

    # Create a temporary requirements file
    temp_requirements_path = 'temp_requirements.txt'
    with open(temp_requirements_path, 'w') as f:
        for package in imports:
            f.write(package + '\n')

    # Install the packages
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', temp_requirements_path])
    os.remove(temp_requirements_path)

# Run the function with the path to the current notebook
current_notebook_path = '/creditfraud/creditfraud/'
install_packages_from_notebook(current_notebook_path)


NameError: name '__file__' is not defined

In [1]:
import lightgbm as lbg
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
import optuna 
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import collections


from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as impipeline
from imblearn.over_sampling import SMOTE
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [27]:
lbg

<module 'lightgbm' from 'c:\\Users\\EpicT\\miniconda3\\envs\\CreditCardFraud1\\Lib\\site-packages\\lightgbm\\__init__.py'>

In [28]:
import sys
!{sys.executable} -m pip install lightgbm

In [29]:
import lightgbm as lgb

Reading the File

In [30]:
df =pd.read_csv('/Kaggle/creditcard.csv')

In [31]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [32]:
df.shape

(284807, 31)

In [33]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

Checking for missing data

In [34]:
alldata = df.isnull().sum().sort_values(ascending =False)
percentage =(df.isnull().sum()/df.isnull().count()*100).sort_values(ascending =False)
pd.concat([alldata, percentage], axis=1, keys=['Total', 'Percent']).transpose()
             

,Time,V16,Amount,V28,V27,V26,V25,V24,V23,V22,...,V10,V9,V8,V7,V6,V5,V4,V3,V2,Class
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
lgb

<module 'lightgbm' from 'c:\\Users\\EpicT\\miniconda3\\envs\\CreditCardFraud1\\Lib\\site-packages\\lightgbm\\__init__.py'>

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [37]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


Checking the Data Unbalance

In [38]:
from sklearn.preprocessing import StandardScaler, RobustScaler
std_scaler = StandardScaler()
rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

df.drop(['Time','Amount'], axis=1, inplace=True)

In [39]:
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

# Amount and Time are Scaled!

df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.783274,-0.994983,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0
1,-0.269825,-0.994983,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0
2,4.983721,-0.994972,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0
3,1.418291,-0.994972,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0
4,0.670579,-0.994960,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0


Cross-validation implemented using stratified sampling ensures that the proportion of the feature of interest is the same across the original data, training set and the test set. This ensures that no value is over/under-represented in the training and test sets, which gives a more accurate estimate of performance/error.

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

print('No Frauds', round(df['Class'].value_counts()[0]/len(df) * 100,2), '% of the dataset')
print('Frauds', round(df['Class'].value_counts()[1]/len(df) * 100,2), '% of the dataset')

X = df.drop('Class', axis=1)
y = df['Class']

Stratkf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

for train_index, test_index in Stratkf.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# We already have X_train and y_train for undersample data thats why I am using original to distinguish and to not overwrite these variables.
# original_Xtrain, original_Xtest, original_ytrain, original_ytest = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the Distribution of the labels


# Turn into an array
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(y_train, return_counts=True)
test_unique_label, test_counts_label = np.unique(y_test, return_counts=True)
print('-' * 100)

print('Label Distributions: \n')
print(train_counts_label/ len(y_train))
print(test_counts_label/ len(y_test))


No Frauds 99.83 % of the dataset
Frauds 0.17 % of the dataset
Train: [ 30473  30496  31002 ... 284804 284805 284806] Test: [    0     1     2 ... 57017 57018 57019]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 30473  30496  31002 ... 113964 113965 113966]
Train: [     0      1      2 ... 284804 284805 284806] Test: [ 81609  82400  83053 ... 170946 170947 170948]
Train: [     0      1      2 ... 284804 284805 284806] Test: [150654 150660 150661 ... 227866 227867 227868]
Train: [     0      1      2 ... 227866 227867 227868] Test: [212516 212644 213092 ... 284804 284805 284806]
----------------------------------------------------------------------------------------------------
Label Distributions: 

[0.99827076 0.00172924]
[0.99827952 0.00172048]


In [41]:
from sklearn.linear_model import LogisticRegression
Regressionmodel = LogisticRegression(random_state=5).fit(X_train, y_train)
Regressionmodel.score(X_test, y_test)


0.9990519829356929

In [42]:
lgb

<module 'lightgbm' from 'c:\\Users\\EpicT\\miniconda3\\envs\\CreditCardFraud1\\Lib\\site-packages\\lightgbm\\__init__.py'>

We have a high accuracy percentage but this is due to the imbalance in datasets so we will use Smote to to compensate for that.

In [43]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV


print('Length of X (train): {} | Length of y (train): {}'.format(len(X_train), len(y_train)))
print('Length of X (test): {} | Length of y (test): {}'.format(len(X_test), len(y_test)))

# List to append the score and then find the average
accuracy_lst = []

# Classifier with optimal parameters
# log_reg_sm = grid_log_reg.best_estimator_
log_reg_sm = LogisticRegression()
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
rand_log_reg = RandomizedSearchCV(LogisticRegression(), log_reg_params, n_iter=4)


# Implementing SMOTE Technique 
# Parameters

for train, test in Stratkf.split(X_train, y_train):
    pipeline = impipeline(SMOTE(sampling_strategy='minority'), rand_log_reg) # SMOTE happens during Cross Validation not before..
    model = pipeline.fit(X_train[train], y_train[train])
    best_est = rand_log_reg.best_estimator_
    prediction = best_est.predict(X_train[test])
    
    accuracy_lst.append(pipeline.score(X_train[test], y_train[test]))
print("accuracy: {}".format(np.mean(accuracy_lst)))

Length of X (train): 227846 | Length of y (train): 227846
Length of X (test): 56961 | Length of y (test): 56961
accuracy: 0.941610351192314


In [44]:
lgb

<module 'lightgbm' from 'c:\\Users\\EpicT\\miniconda3\\envs\\CreditCardFraud1\\Lib\\site-packages\\lightgbm\\__init__.py'>

In [64]:

def objective(trial):
    smote= SMOTE(random_state=98)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    train_data = lgb.Dataset(X_train_res, label=y_train_res)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


    boosting_type = trial.suggest_categorical('boosting_type', ['gbdt', 'dart', 'goss'])
    num_leaves=trial.suggest_int('num_leaves', 10, 100)
    learning_rate =trial.suggest_loguniform('learning_rate',0.01, 0.1)

    params = { 
    'learning_rate': learning_rate,
    'boosting_type': boosting_type,
    'num_leaves': num_leaves,
    'n_estimators': 100,
    'objective': 'binary',
    'verbose': -1,
    'metric': 'auc'
    }

    gbm = lgb.train(params, train_data, num_boost_round=200, valid_sets=[test_data], early_stopping_min_delta=10)
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    auc = roc_auc_score(y_test, y_pred)
    # Evaluating AUC on validation set
    y_pred_val = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    auc_val = roc_auc_score(y_test, y_pred_val)

    #print metrics to monitor for overfitting
    print(f'Training AUC: {auc_train:.4f}')
    print(f'Validation AUC: {auc_val:.4f}')

    return auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

#Obtaining the best trial
best_trial = study.best_trial
#Training the model using the parameters found by optuna
smote=SMOTE(random_state=98)
X_train_res, y_train_res = smote.fit_resample(X_train,y_train)
train_data =lgb.Dataset(X_train_res, label=y_train_res)
test_data =lgb.Dataset(X_test, label=y_test, reference=train_data)

best_params=best_trial.params
best_params['verbose'] = -1
gbm= lgb.train(best_params, train_data, num_boost_round=20, valid_sets=[test_data])
y_pred =gbm.predict (X_test, num_iteration=gbm.best_iteration)

y_pred_binary = np.where(y_pred > 0.5, 1, 0)


print(classification_report(y_test, y_pred_binary))
print(confusion_matrix(y_test, y_pred_binary))



[I 2024-07-01 14:37:08,422] A new study created in memory with name: no-name-fe7f12c0-63fe-4df0-afe4-913a1112ce90
[W 2024-07-01 14:37:08,686] Trial 0 failed with parameters: {'boosting_type': 'gbdt', 'num_leaves': 87, 'learning_rate': 0.05362366885338611} because of the following error: TypeError("train() got an unexpected keyword argument 'early_stopping_min_delta'").
Traceback (most recent call last):
  File "c:\Users\EpicT\miniconda3\envs\CreditCardFraud1\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\EpicT\AppData\Local\Temp\ipykernel_3044\3459253187.py", line 23, in objective
    gbm = lgb.train(params, train_data, num_boost_round=200, valid_sets=[test_data], early_stopping_min_delta=10)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: train() got an unexpected keyword argument 'early_stopping_min_de

TypeError: train() got an unexpected keyword argument 'early_stopping_min_delta'